In [3]:
!pip install scipy
!pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV


## 1: Obtener el dataset

In [5]:
# Obtener dataset del csv
CSV= pd.read_csv('PCA_datos.csv')
X_or = CSV['0']
Y_or = CSV['Complains']
Y_or.shape

(2836,)

## 2: dividir en conjuntos de entrenamiento y test

In [6]:
x_train, x_test, y_train, y_true = train_test_split(X_or,Y_or, test_size=0.3)

In [7]:
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
# y_train = pd.DataFrame(y_train)
# y_true = pd.DataFrame(y_true)

In [8]:
print("el tamaño de x es {}, el tamaño de y es {}".format(x_train.shape, y_train.shape))
print("el tamaño de x_test es {}, el tamaño de y_true es {}".format(x_test.shape, y_true.shape))

el tamaño de x es (1985, 1), el tamaño de y es (1985,)
el tamaño de x_test es (851, 1), el tamaño de y_true es (851,)


# KNN

## 3: Entrenar y ejecutar el modelo

In [9]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [10]:
y_pred_knn = knn.predict(x_test)

## 4: Evaluar el modelo

In [11]:
confusion_matrix(y_true, y_pred_knn)

array([[775,  12],
       [ 61,   3]])

In [12]:
puntuacion_knn_antigua = accuracy_score(y_true, y_pred_knn)
print(puntuacion_knn_antigua)

0.9142185663924794


In [13]:
puntuacion_knn_balanceada_antigua =balanced_accuracy_score(y_true, y_pred_knn)
print(puntuacion_knn_balanceada_antigua)

0.5158136118170267


En este caso, es mejor usar la puntuación balanceada, ya que hay muchos más valores de 0 (sin quejas) que de 1.
Esto lleva al KNN a confundirse y tener muchos falsos negativos a la hora de detectar quejas como se ve en la matriz de confusión.

## 5: Optimizar K

In [14]:
clf = GridSearchCV(KNeighborsClassifier(),scoring='balanced_accuracy',param_grid={'n_neighbors':[2,3,4,5,6,7,8,15,16,31,32]})

clf.fit(pd.DataFrame(X_or),Y_or)
df_resultados_grid = pd.DataFrame(clf.cv_results_)
df_resultados_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003039,0.000911,0.030230,0.005386,2,{'n_neighbors': 2},0.504165,0.499040,0.501273,0.489443,0.506071,0.499998,0.005800,8
1,0.002424,0.000416,0.027580,0.002940,3,{'n_neighbors': 3},0.497460,0.516941,0.502545,0.511495,0.503192,0.506327,0.006959,1
2,0.003174,0.000646,0.028301,0.002071,4,{'n_neighbors': 4},0.505122,0.500000,0.492322,0.500313,0.509910,0.501534,0.005861,3
3,0.002441,0.000285,0.027537,0.003872,5,{'n_neighbors': 5},0.513118,0.498081,0.499353,0.507344,0.509910,0.505561,0.005894,2
4,0.001984,0.000126,0.022167,0.001171,6,{'n_neighbors': 6},0.509912,0.499040,0.495202,0.495202,0.500000,0.499871,0.005388,9
5,0.002032,0.000133,0.021032,0.000257,7,{'n_neighbors': 7},0.509912,0.498081,0.492322,0.491363,0.500000,0.498336,0.006658,11
6,0.002255,0.000283,0.021945,0.000983,8,{'n_neighbors': 8},0.500000,0.500000,0.499040,0.499040,0.500000,0.499616,0.000470,10
7,0.002065,0.000139,0.022687,0.000979,15,{'n_neighbors': 15},0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,4
8,0.001950,0.000085,0.021411,0.000220,16,{'n_neighbors': 16},0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,4
9,0.002061,0.000231,0.021787,0.000239,31,{'n_neighbors': 31},0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,4


In [15]:
clf.best_estimator_

KNeighborsClassifier(n_neighbors=3)

Con estos resultados, queda probado que el mejor estimador entre los posibles k escogidos es k=3

## 6: Reentrenar KNN con el k seleccionado

In [16]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)

In [17]:
confusion_matrix(y_true, y_pred_knn)

array([[769,  18],
       [ 60,   4]])

In [18]:
puntuacion_knn_nueva = accuracy_score(y_true, y_pred_knn)

In [19]:
puntuacion_knn_balanceada_nueva = balanced_accuracy_score(y_true, y_pred_knn)

In [20]:
print('La puntuación sin balancear ha pasado de {0} a {1}'.format(puntuacion_knn_antigua, puntuacion_knn_nueva))
print('La puntuación balanceada ha pasado de {0} a {1}'.format(puntuacion_knn_balanceada_antigua, puntuacion_knn_balanceada_nueva))


La puntuación sin balancear ha pasado de 0.9142185663924794 a 0.9083431257344301
La puntuación balanceada ha pasado de 0.5158136118170267 a 0.51981416772554


Como estábamos priorizando la puntuación balanceada, al haber mejora en esta, podemos decir que el modelo ha mejorado.

# Árboles de decisión

In [21]:
from sklearn.tree import DecisionTreeClassifier


## 3: Entrenar y ejecutar el modelo

In [22]:
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [23]:
y_pred_arbol = clf.predict(x_test)

## 4: Evaluar el modelo

In [24]:
confusion_matrix(y_true, y_pred_arbol)

array([[725,  62],
       [ 51,  13]])

In [25]:
puntuacion_arbol_antigua = accuracy_score(y_true, y_pred_arbol)
print(puntuacion_arbol_antigua)

0.8672150411280846


In [26]:
puntuacion_arbol_balanceada_antigua = balanced_accuracy_score(y_true, y_pred_arbol)
print(puntuacion_arbol_balanceada_antigua)

0.5621724110546379


Estamos ante el mismo caso del KNN

## 5: Optimizar el árbol

In [27]:
clf = GridSearchCV(DecisionTreeClassifier(),scoring='balanced_accuracy',param_grid={'max_depth':[2,3,4,5,6,7,8,9,10,15,30,90,270]})

In [28]:
clf.fit(pd.DataFrame(X_or),Y_or)
df_resultados_grid = pd.DataFrame(clf.cv_results_)
df_resultados_grid

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006106,0.000594,0.004986,0.000634,2,{'max_depth': 2},0.500000,0.500000,0.500000,0.510870,0.498081,0.501790,0.004600,4
1,0.004908,0.001439,0.003421,0.000995,3,{'max_depth': 3},0.498084,0.498081,0.500000,0.500000,0.498081,0.498849,0.000940,9
2,0.004157,0.000451,0.002835,0.000838,4,{'max_depth': 4},0.498084,0.497121,0.507990,0.498081,0.498081,0.499871,0.004077,8
3,0.006128,0.000994,0.003348,0.000846,5,{'max_depth': 5},0.498084,0.497121,0.507990,0.499040,0.498081,0.500063,0.004010,6
4,0.004434,0.000111,0.002463,0.000106,6,{'max_depth': 6},0.498084,0.503192,0.507031,0.499040,0.494242,0.500318,0.004401,5
5,0.004271,0.000297,0.002145,0.000189,7,{'max_depth': 7},0.507038,0.503192,0.504152,0.493282,0.492322,0.499997,0.006017,7
6,0.005124,0.000346,0.002763,0.000654,8,{'max_depth': 8},0.505122,0.499353,0.504152,0.490403,0.492322,0.498271,0.006000,10
7,0.004686,0.000269,0.002077,0.000091,9,{'max_depth': 9},0.502249,0.494555,0.500313,0.484645,0.491363,0.494625,0.006336,11
8,0.004628,0.000100,0.001978,0.000091,10,{'max_depth': 10},0.500333,0.494555,0.498394,0.481766,0.489443,0.492898,0.006695,13
9,0.004826,0.000206,0.001992,0.000264,15,{'max_depth': 15},0.520490,0.488797,0.494868,0.466411,0.502545,0.494622,0.017674,12


In [29]:
clf.best_estimator_

DecisionTreeClassifier(max_depth=30)

## 6: Reentrenar el árbol con el max_depth seleccionado

In [30]:
clf = DecisionTreeClassifier(max_depth=30)
clf.fit(x_train, y_train)
y_pred_arbol = clf.predict(x_test)

In [31]:
confusion_matrix(y_true, y_pred_arbol)

array([[725,  62],
       [ 51,  13]])

In [32]:
puntuacion_arbol_nueva = accuracy_score(y_true, y_pred_arbol)

In [33]:
puntuacion_arbol_balanceada_nueva = balanced_accuracy_score(y_true, y_pred_arbol)

In [34]:
print('La puntuación sin balancear ha pasado de {0} a {1}'.format(puntuacion_arbol_antigua, puntuacion_arbol_nueva))
print('La puntuación balanceada ha pasado de {0} a {1}'.format(puntuacion_arbol_balanceada_antigua, puntuacion_arbol_balanceada_nueva))

La puntuación sin balancear ha pasado de 0.8672150411280846 a 0.8672150411280846
La puntuación balanceada ha pasado de 0.5621724110546379 a 0.5621724110546379


# Comparación de resultados

In [35]:
print('La puntuación sin balancear del KNN ha pasado de {0} a {1}'.format(puntuacion_knn_antigua, puntuacion_knn_nueva))
print('La puntuación balanceada del KNN ha pasado de {0} a {1}'.format(puntuacion_knn_balanceada_antigua, puntuacion_knn_balanceada_nueva))
print('La puntuación sin balancear del árbol ha pasado de {0} a {1}'.format(puntuacion_arbol_antigua, puntuacion_arbol_nueva))
print('La puntuación balanceada del árbol ha pasado de {0} a {1}'.format(puntuacion_arbol_balanceada_antigua, puntuacion_arbol_balanceada_nueva))

La puntuación sin balancear del KNN ha pasado de 0.9142185663924794 a 0.9083431257344301
La puntuación balanceada del KNN ha pasado de 0.5158136118170267 a 0.51981416772554
La puntuación sin balancear del árbol ha pasado de 0.8672150411280846 a 0.8672150411280846
La puntuación balanceada del árbol ha pasado de 0.5621724110546379 a 0.5621724110546379


En este caso el KNN ha tenido una mejora de resultados más pronunciada y tuvo los mejores resultados